<a id="top"></a>
# ULLYSES Time-Series Spectral Products

***

## Learning Goals

By the end of this tutorial, you will:

- learn how the ULLYSES team creates time-series spectra
- learn how to create generic time-series spectra 

## Table of Contents
**0. [Introduction](#introduction)**

**1. [Obtain Data](#sec1)**

**2. [Creating a time-series spectrum](#sec2)**

\- 2.1 [Time-series YAML files](#sec21)

\- 2.2 [Running the time-series creation routine](#sec22) 

**3. [Time-series Spectrum Format](#sec3)**

**4. [Creating your own custom STIS spectra](#sec4)**

<a id="introduction"></a>
## Introduction

The Hubble Space Telescope’s (HST) Ultraviolet Legacy Library of Young Stars as Essential Standards ([ULLYSES](https://ullyses.stsci.edu/index.html)) program has devoted approximately 1,000 HST orbits to the production of an ultraviolet spectroscopic library of young high- and low-mass stars in the local universe. The ULLYSES team produces several types of High Level Science Products (HLSPs). Products are made using HST, FUSE, and LCO data.

For the vast majority of data, the ULLYSES team [coadds](https://ullyses.stsci.edu/ullyses-data-description.html#CoaddSpectra) all data of the same instrument and grating combination. However, for targets who exhibit variable flux, coaddition should not be performed. Instead, the ULLYSES team creates [time-series spectra](https://ullyses.stsci.edu/ullyses-data-description.html#hlspFormatTimeSpec), or TSS. 

Time-series spectra (TSS) come in two flavors:
1. exposure level, where each input spectrum used as-is (with suffix `_tss.fits`)
2. sub-exposure level, where each input spectrum is split into *sub-exposures*, to probe a finer timescale than the executed exposure time (with suffix `_split-tss.fits`)

These products are created from HST spectroscopic data as well as imaging data from the Las Cumbres Observatory (LCO).
The products available for each type of target and data are summarized in the table below. TSS are all level 5 data products.

| Target         | Data    | TSS Type                        |
| -------------- | ------- | ------------------------------- |
| Survey TTS     | LCO     | Exposure level                  |
| Survey TTS     | HST     | Exposure level                  |
| Monitoring TTS | LCO     | Exposure level                  |
| Monitoring TTS | HST     | Exposure and sub-exposure level |

In this notebook we will show how exposure-level TSS are created, and how to create them yourself.

### Imports
We need only access to basic python packages, `astropy` for reading FITS files, `matplotlib` for plotting, and ULLYSES packages (`ullyses` and `ullyses_utils`) to create time-series spectra.

In [ ]:
import sys
current_stdout = sys.stdout
import os
import glob
import numpy as np
import pprint
from collections import Counter

from astropy.io import fits

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize']=12,5
#plt.style.use('seaborn-v0_8-notebook')
plt.style.use('tableau-colorblind10')
#%matplotlib inline
%matplotlib widget

import ullyses_utils
from ullyses_utils.readwrite_yaml import read_config
from ullyses.timeseries_wrapper import exp_star

***

<a id="sec1"></a>
## Obtain Data

To download data, we will retrieve it directly from MAST. For this example, we'll download 8 COS/G160M observations of ULLYSES survey T Tauri Star CVSO-109. We will only retrieve the final 1D spectra (`x1d` files). All files use the G160M cenwave 1611 setting.

These datasets will download into a new directory in your current working directory, called `notebook_download/`.

In [ ]:
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LCCP03010%2Flccp03r8q_x1d.fits" --output "notebook_download/lccp03r8q_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LCCP03010%2Flccp03raq_x1d.fits" --output "notebook_download/lccp03raq_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LCCP03010%2Flccp03rcq_x1d.fits" --output "notebook_download/lccp03rcq_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LCCP03010%2Flccp03req_x1d.fits" --output "notebook_download/lccp03req_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LE9K2C020%2Fle9k2cf4q_x1d.fits" --output "notebook_download/le9k2cf4q_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LE9K2C020%2Fle9k2cf6q_x1d.fits" --output "notebook_download/le9k2cf6q_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LE9K2C020%2Fle9k2cf8q_x1d.fits" --output "notebook_download/le9k2cf8q_x1d.fits" --fail --create-dirs
!curl -L -X GET "https://mast.stsci.edu/search/hst/api/v0.1/retrieve_product?product_name=LE9K2C020%2Fle9k2cfaq_x1d.fits" --output "notebook_download/le9k2cfaq_x1d.fits" --fail --create-dirs

***

<a id="sec2"></a>
## Inspect 1D spectra

Let's take a look at the MAST 1D spectra for these eight datasets.

In [ ]:
# Get all x1ds
x1ds = glob.glob("notebook_download/lccp03*x1d.fits") + glob.glob("notebook_download/le9k2c*x1d.fits")
# Now sort them by observation start date
mjdstarts = [fits.getval(x, "EXPSTART", 1) for x in x1ds]
inds_sorted = np.argsort(mjdstarts)
x1ds = np.array(x1ds)[inds_sorted]

In [ ]:
# Function for plotting 1D spectra
def plot_x1ds(x1ds, xlim=None):
    """
    Plot x1d spectra.
    Args:
        x1ds (array-like): List of x1d files to plot.
        xlim (array-like): (Optional) If specified, x limits of plot.
    """
    # We will cycle through the colors in a colormap.
    fig = plt.figure()
    cmap = matplotlib.colormaps["viridis"]
    evals = np.linspace(0, 1, len(x1ds))
    # To keep track of legend items
    plot_legends = []
    for i,item in enumerate(x1ds):
        # Get ext=1 SCI data and observation date
        data = fits.getdata(item)
        date_obs = fits.getval(item, "date-obs", 1)
        time_obs = fits.getval(item, "time-obs", 1)
        color = cmap(evals[i])
        # Plot flux vs. wavelength
        for j in range(len(data["flux"])): # This is to loop over the segments used.
            plot_legend, = plt.plot(data["wavelength"][j], data["flux"][j], color=color, alpha=0.6, 
                     label=f"{os.path.basename(item)} {date_obs} {time_obs}")
        plot_legends.append(plot_legend)
        # Optionally set x limits
        if xlim is not None:
            plt.xlim(xlim[0], xlim[1])
        plt.title("COS/G160M spectra - CVSO-109")
        plt.xlabel("Wavelength [$\mathrm{\AA}$]")
        plt.ylabel("Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]")
    plt.legend(handles=plot_legends)

In [ ]:
plot_x1ds(x1ds)

It's a little hard to see any variability here, so let's zoom in on the C IV line.

In [ ]:
plot_x1ds(x1ds, xlim=(1545, 1555))

From this plot, we can see there seems to be two different flux states for this star, corresponding to the different observational epochs in 2014 and 2020. Because of this, the ULLYSES team cannot coadd all of the available G160M data. Instead, we create a time-series spectrum for this target.

<a id="sec2"></a>
## Creating a time-series spectrum

<a id="sec21"></a>
### Time-series YAML files

To create a TSS, we need to specify which grating and datasets should be used. This is done by creating a YAML file with the relevant info.
YAML files are convenient, human-readable files that act as configuration setups. The ULLYSES team creates timeseries YAML filesfor all T Tauri stars that show variation across different observational epochs

Let's take a look at the CVSO-109 timeseries YAML file now. All timeseries YAML files can be found in the [`ullyses_utils` package](https://github.com/spacetelescope/ullyses-utils/tree/main/src/ullyses_utils/data/timeseries).

In [ ]:
# Find your local installation of the ullyses_utils package
utils_dir = ullyses_utils.__path__[0]
# Get the path to the timeseries YAML files
yamls = glob.glob(os.path.join(utils_dir, f"data/timeseries/*cvso-109*.yaml"))
yamls.sort()
pprint.pprint(yamls)

There is just one YAML for CVSO-109 COS data, so let's open it and take a look.

In [ ]:
# Use the ULLYSES convenience function to read the YAML file
yaml = yamls[0]
config = read_config(yaml)
print(f"{os.path.basename(yaml)}:")
for k,v in config.items():
    print(f"    {k}: {v}")

Let's walk through what's contained in this file, but keep in mind CVSO-109 is a *survey* T Tauri star.
- `exp_tss` and `sub_exp_tss` (Bool): Time-series spectra of the specified type(s) will be created based on these values.
- `observatory`, `instrument`, and `gratings` (str): Describe the mode(s) for which TSS should be created.
- `bins` (dict): This is only used when creating sub-exposure level TSS, which we do not create for survey stars.
- `good_files` (list): These are files that should be used to create the TSS. Using the normal [HST IPPPSSOOT](https://archive.stsci.edu/hlsp/ipppssoot.html) naming convention, either the IPPPSS or IPPPSSOOT may be supplied.
- `bad_files` (list): If specified, we can list any IPPPSSOOT which should not be included in the TSS. This is usually used for monitoring stars only.
- `wavelength_shift` (dict): If any input COS spectra suffer from wavelength offsets (e.g. from miscentering in the aperture Along-Dispersion direction), the shift files used to correct them may be specified here.

<a id="sec22"></a>
### Running the time-series creation routine

To create a time-series spectrum, we need to run a function called `exp_star`. This is in the `ullyses.timeseries_wrapper` module. The following information is accepted at runtime:

**REQUIRED**

- `datadir` (str): The path the original data will be copied to, to avoid overwriting original 1D spectra.
- `orig_datadir` (str): The path to the data to be turned into a time-series spectrum.
- `tss_outdir` (str): The path that the time-series product will be written to.
- `targ` (str): The target name. The code is designed to work on ULLYSES targets, and timeseries YAML files can be automatically located if the "ULLYSES HLSP" name is provided. These names can be found in the [ULLYSES alias file](https://github.com/spacetelescope/ullyses-utils/blob/main/src/ullyses_utils/data/target_metadata/ullyses_aliases.csv).

**OPTIONAL**

- `yamlfile` (str): If the target name does not exactly match the ULLYSES HLSP name, or if you want to specify a custom YAML file, it must be explicitly supplied here.
- `min_exptime` (float): The minimum exposure time in seconds, above which, datasets will be included in the output TSS. The default is 0.1s.
- `instrument` (str): Sometimes, for a single star, there are multiple YAML files for mulitple instruments. In this case, the instrument of interest must be supplied. The default is COS.

Now, let's see how we can run the wrapper for CVSO-109, which can be done in two different ways. Both methods are equivalent.

In [ ]:
# Let the code automatically find the YAML file
exp_star(datadir="notebook_download/tss_data", orig_datadir="notebook_download/", tss_outdir="tss_output",
         targ="cvso-109", instrument="cos")

# Manually specify the yamlfile
#exp_star(datadir="notebook_download/tss_data", orig_datadir="notebook_download/", tss_outdir="tss_output",
#         targ="cvso-109", yamlfile=yaml)

From the output, we can see that the code copied the original data (from `orig_datadir`) to a new directory (`datadir`). 
It then added all the matching x1ds specified in the YAML file to a list, then inserted each one into the time-series spectrum, in order of observation start time. Finally, it wrote a timeseries product with suffix `_tss.fits`. Now let's explore that output file.

<a id="sec3"></a>
## Time-series Spectrum Format

Time-series spectra essentially stack all individual 1D spectral fluxes and errors into a 2D array. All input data are rebinned onto the same wavelength grid. Data are then assembled into a 2-D array by inserting each rebinned spectrum into a row of a 2-D image, where the rows are ordered in time. 

Flux and error arrays are 2D arrays with wavelength increasing along X, and time increasing along Y. The wavelength values for each column of the 2D data are stored in the wavelength array, while the mjdstart and mjdend columns store the start and end times for each row of the flux and error arrays.

Let's look at the TSS we just created to further understand the format.

In [ ]:
tss = "tss_output/hlsp_ullyses_hst_cos_cvso-109_g160m_dr7_tss.fits"
fits.info(tss)

This is the typical format of ULLYSES HLSPs: a primary header, a SCIENCE 1st extension, and a PROVENANCE 2nd extension that lists key metadata for each contributing 1D spectrum. For more info on the HLSP format, see the [Walkthrough notebook](https://github.com/spacetelescope/ullyses/blob/main/notebooks/ullyses-walkthrough.ipynb). Let's look at the SCIENCE extension.

In [ ]:
data = fits.getdata(tss)
data.columns

We can tell a lot about the format of this file just from the SCIENCE array dimensions: 
- the flux and error arrays are 31,832 rows by 8 columns
- the mjdstart and mjdend arrays are both 8 rows
- the wavelength array is 31,832 rows

This is because the the X axis of the 2D arrays (flux and error) represents wavelength, while the Y axis represents time (mjdstart/end). Let's plot some data to really illustrate this.

In [ ]:
# The wavelength is the same for all stacked spectra, so we only need to read it in once
wl = data["wavelength"][0]
cmap = matplotlib.colormaps["viridis"]
numspectra = len(data["flux"][0])
evals = np.linspace(0, 1, numspectra)
# Store the PROVENANCE extension
prov = fits.getdata(tss, 2)
fig = plt.figure()
for i in range(numspectra):
    # Get flux for that individual time index/spectrum
    flux = data["flux"][0][i] # This is an array
    # Get the corresponding start time
    mjdstart = data["mjdstart"][0][i] # This is a single value
    # Get the corresponding filename from the PROVENANCE extension
    filename = prov["filename"][i]
    color = cmap(evals[i])
    
    # Plot flux vs. wavelength
    plt.plot(wl, flux, color=color, alpha=0.6, label=f"{filename} {mjdstart}")
    plt.xlim(1545, 1555)
    plt.legend()
    plt.title("COS/G160M time-series spectrum - CVSO-109")
    plt.xlabel("Wavelength [$\mathrm{\AA}$]")
    plt.ylabel("Flux\n[ergs/s/cm^2/$\mathrm{\AA}$]")

This looks very similar to the plot we made before, just by reading in the individual x1ds... In fact, let's plot that up again for comparison.

In [ ]:
plot_x1ds(x1ds, xlim=(1545, 1555))

As expected, they are identical! 

The benefit to using time-series spectra is that the ULLYSES team has already gone through the work of packaging up all relevant spectra into a single file for your use. For this target, there were only 8 contributing spectra, but for the monitoring stars, exposure level TSS can have over 100 contributing spectra. And for sub-exposure TSS, there can be over 700 individual sub-exposures in a single file! Additionally, any manual corrections required for each contributing 1D spectrum (COS/NUV vignetting, STIS defringing, custom extraction, wavelength shifts, etc.) are incorporated in ULLYSES TSS.

<a id="sec4"></a>
## Creating your own custom time-series spectra

To create your own custom time-series spectra, you must first create new input YAML files as required by the ULLYSES code. You can find a [template form](https://github.com/spacetelescope/ullyses-utils/blob/main/src/ullyses_utils/data/timeseries/tss_template.yaml) in the same `ullyses_utils` repo, as shown below. You would simply need to modify specific parameters as required for your data. The name of the YAML file must follow the format `<target>_<instrument>.yaml`, and be lowercase, in order to be properly processed *automatically* (the target must be a ULLYSES target)

The template is shown below, along with comments that explain various parameters.

In [ ]:
utils_dir = ullyses_utils.__path__[0]
template = os.path.join(utils_dir, f"data/timeseries/tss_template.yaml")
with open(template, "r") as f:
    contents = f.read()
    print(contents)

***

## Additional Resources

- [ULLYSES](https://ullyses.stsci.edu)
- [MAST API](https://mast.stsci.edu/api/v0/index.html)

## About this Notebook
For support, contact us at the [ULLYSES Helpdesk](https://stsci.service-now.com/hst?id=sc_cat_item&sys_id=a3b8ec5edbb7985033b55dd5ce961990&sysparm_category=ac85189bdb4683c033b55dd5ce96199c).

**Author:**  Jo Taylor \
**Updated On:** March 7, 2024

## Citations
* See the [ULLYSES website](https://ullyses.stsci.edu/ullyses-cite.html) for citation guidelines.


***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 